<a href="https://colab.research.google.com/github/ElizavetaNosova/CompLing_homeworks/blob/master/Topicmodeling_gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install gensim
!pip install pandas
!pip install nltk
!pip install pymorphy2[fast]

     |████████████████████████████████| 51kB 3.9MB/s 
     |████████████████████████████████| 7.1MB 25.3MB/s 
     |████████████████████████████████| 256kB 46.1MB/s 
  Created wheel for DAWG: filename=DAWG-0.7.8-cp36-cp36m-linux_x86_64.whl size=775052 sha256=8218932c15f11c6866dcbb080d084dc64cf95142870b27dd485909bb167c9159
  Stored in directory: /root/.cache/pip/wheels/d4/88/d0/4e4abc83eb8f59a71e8dbd8ba99fd5615a3af1fac1ef7f8825
Successfully built DAWG


In [0]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 8.2MB/s 
     |████████████████████████████████| 552kB 47.1MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=55f552ab7090b01e59e6417f9d06ca364f31d30f547f84409265acb290241c96
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=e4508edc3ff8a676c3c5d436170b49947e2bd0773d45cfa43afa67db7f57a4b9
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [0]:
import gensim
import json
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

morph = MorphAnalyzer()

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

При нормализации сразу не будем включать слова из списка стоп-слов и другие слова служебных частей речи, местоимения (видимо, местоимения-существительные, так как pymorphy только их выделяет отдельно) и числительные.
Полагаю, ято в связи с этим список стопслов можно изначально урезать: скопировать из nltk и оставить только другие части речи, раз вышеупомянутые всё равно будут проверяться отдельно.
Впрочем, непонятно, зачем я это делаю, если эти слова всё равно вряд ли станут "ключевыми" для тем

In [0]:
stops = ['как','так', 'было', 'еще', 'нет', 'теперь', 'когда', 'вдруг', 'уже', 'быть', 'был', 'нибудь', 'опять', 'там', 'потом', 'себя', 'ничего', 'может',  'тут', 'где', 'есть', 'надо', 'чем', 'была', 'сам', 'раз', 'тоже', 'себе', 'под', 'будет', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве',  'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между']

def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def opt_normalize(texts, n):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(n) if morph.parse(word)[0].tag.POS not in ['NUMR', 'NPRO', 'PREP', 'CONJ', 'PRCL', 'INTJ']}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    words = [word.strip(string.punctuation) for word in text.split()]
    words = [word for word in words if word]
    
    return words

In [0]:
from google.colab import files

uploaded = files.upload()

Saving habr_texts.txt to habr_texts (1).txt


In [0]:
texts = open('habr_texts.txt').read().splitlines()
texts = opt_normalize([tokenize(remove_tags(text.lower())) for text in texts], 30000)

In [0]:
print(len(texts))

4121


Подберём параметры, при которых когерентность будет наибольшей. Для начала поберём параметры нграм и словаря при каких-нибудь параметрах модели. Будем надеяться, что при других параметрах модели они тоже хорошие

In [0]:
ngram_threshold_values = [0.2, 0.4, 0.6, 0.8]
no_above_values = [0, 0.05, 0.1, 0.2, 0.25]
no_below_values = [10, 20, 30, 50]
best_res = 0
best_params = {}
for ngram_threshold in ngram_threshold_values:
  for no_above in no_above_values:
    for no_below in no_below_values:
      try:
        ph = gensim.models.Phrases(texts, scoring='npmi', threshold=ngram_threshold) # threshold можно подбирать
        p = gensim.models.phrases.Phraser(ph)
        ngrammed_texts = p[texts]
        dictinary = gensim.corpora.Dictionary(texts)
        dictinary.filter_extremes(no_above=no_above, no_below=no_below)
        dictinary.compactify()
        corpus = [dictinary.doc2bow(text) for text in texts]
        lda = gensim.models.LdaMulticore(corpus, 100, id2word=dictinary, eval_every=0)
        coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                    texts=texts, 
                                                     dictionary=dictinary, coherence='c_v')
        if coherence_model_lda.get_coherence() > best_res:
          best_res = coherence_model_lda.get_coherence()
          best_params['ngram_threshold'] = ngram_threshold
          best_params['no_above'] = no_above
          best_params['no_below'] = no_below
      except Exception as e:
         print(e)
print(best_params)


cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)


Process ForkPoolWorker-56:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
  File "/usr/local/lib/python3.6/dist-packages/gensim/models/ldamulticore.py", line 333, in worker_e_step
    worker_lda.do_estep(chunk)  # TODO: auto-tune alpha?
  File "/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py", line 725, in do_estep
    gamma, sstats = self.inference(chunk, collect_sstats=True)
  File "/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py", line 679, in inference
    phinorm = np.dot(expElogthetad, expElogbetad) + eps
  File "<__array_function__ internals>", line 6, in dot
KeyboardInterrupt


KeyboardInterrupt: ignored

Если честно, все варианты я не перебрала, но через какое-то время (часа полтора) лучшим результатом было {'ngram_threshold': 0.8, 'no_above': 0.05, 'no_below': 10}, когерентность 0.3816700738434562.
Как минимум все предложенные характеристики нграм хотя бы по разу были опробованы, раз дело дошло до 0.8
 Промежуточные выводы: 
1. некоторые параметры отсекают весь корпус (отлавливалась соответствующая ошибка)
2. не надо перебирать 80 сочетаний характеристик, если я хочу, чтобы процесс закончился через адекватное время

In [0]:
print(best_params) 
print(best_res)

{'ngram_threshold': 0.8, 'no_above': 0.05, 'no_below': 10}
0.3816700738434562


С учётом второго вывода  подберём настройки модели. Очень хочется верить, что, если корпус будет оформлен один раз, процесс будет хоть немного быстрее.

In [0]:
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.8) 
p = gensim.models.phrases.Phraser(ph)
nrammed_texts = p[texts]
dictinary = gensim.corpora.Dictionary(texts)
dictinary.filter_extremes(no_above=0.05, no_below=10)
dictinary.compactify()
corpus = [dictinary.doc2bow(text) for text in texts]

In [0]:
num_topics = [50, 100, 200]
n_eval = [0,1]
best_res = 0
best_params = {}
best_topics = ''
for n_topics in num_topics:
  for n in n_eval:
      lda = gensim.models.LdaMulticore(corpus, n_topics, id2word=dictinary, eval_every=n)
      coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                texts=texts, 
                                                  dictionary=dictinary, coherence='c_v')
      if coherence_model_lda.get_coherence() > best_res:
        best_res = coherence_model_lda.get_coherence()
        best_params['topics'] = n_topics
        best_params[''] = n
        best_model = lda

In [0]:
best_model.print_topics()

[(34,
  '0.019*"инцидент" + 0.013*"фоновый" + 0.011*"client" + 0.010*"chrome" + 0.009*"вкладка" + 0.009*"учётный" + 0.008*"бюджет" + 0.007*"иб" + 0.007*"долг" + 0.006*"select"'),
 (80,
  '0.034*"twitter" + 0.019*"твит" + 0.018*"бот" + 0.012*"аккаунт" + 0.008*"выборка" + 0.008*"ботнет" + 0.008*"тестировщик" + 0.007*"общественный" + 0.006*"северный" + 0.006*"streaming"'),
 (38,
  '0.036*"боль" + 0.032*"китай" + 0.015*"правительство" + 0.015*"атрибут" + 0.015*"vpn" + 0.009*"сопротивление" + 0.008*"•" + 0.007*"lt;/summary&gt" + 0.007*"чиновник" + 0.007*"англ"'),
 (66,
  '0.014*"клетка" + 0.009*"атрибут" + 0.007*"математика" + 0.007*"женщина" + 0.006*"аудио" + 0.006*"образование" + 0.006*"ген" + 0.005*"хакер" + 0.005*"ит" + 0.005*"семья"'),
 (98,
  '0.026*"foreach" + 0.019*"итерация" + 0.018*"do" + 0.018*"apply" + 0.014*"вектор" + 0.013*"fun" + 0.009*"матрица" + 0.009*"repeat" + 0.008*"result" + 0.008*"margin"'),
 (61,
  '0.040*"домен" + 0.012*"переадресация" + 0.012*"миграция" + 0.008*"seo

Лучшие темы:
- ситуация в Китае (даже если убрать Китай, сочетание правительства и боли прекрасно)
(38,
  '0.036*"боль" + 0.032*"китай" + 0.015*"правительство" + 0.015*"атрибут" + 0.015*"vpn" + 0.009*"сопротивление" + 0.008*"•" + 0.007*"lt;/summary&gt" + 0.007*"чиновник" + 0.007*"англ"'),

- экраны и графика:
 '0.020*"яркость" + 0.014*"стикер" + 0.014*"репликация" + 0.007*"гаджет" + 0.006*"no" + 0.006*"more" + 0.005*"атрибут" + 0.005*"координата" + 0.005*"выделение" + 0.005*"цветок"'),

 - биоинформатика
 51,
  '0.014*"клетка" + 0.012*"временный" + 0.007*"ген" + 0.006*"php" + 0.005*"виртуализация" + 0.005*"broadcast" + 0.005*"прерывание" + 0.005*"малый" + 0.005*"образование" + 0.004*"параллельный"'),
 



В выдачу попадало много слов типа "user" и команд языков программирования. В принципе, это не помешало, но перед следующим этапом лучше эти слова исключить. Добавим в предобработку ограничение: будем учитывать только слова, написанные кириллицей.
Ещё попало имя "Андрей". В идеале стоило бы скопировать словарь имён и стереть оттуда имена известных чат-ботов и т.п., но будем надеяться, что корреляция имён с темами маловероятна.

In [0]:
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/habr_texts.txt.zip

--2020-02-06 12:04:08--  https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/habr_texts.txt.zip
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/habr_texts.txt.zip [following]
--2020-02-06 12:04:08--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/habr_texts.txt.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18645068 (18M) [application/zip]
Saving to: ‘habr_texts.txt.zip’

habr_texts.txt.zip  100%[===================>]  17.78M  80.1MB/s    in 0.2s    

2020-02-06 12:04:09 (80.1 MB/s) - ‘habr_texts.t

In [0]:
!unzip habr_texts.txt.zip

Archive:  habr_texts.txt.zip
  inflating: habr_texts.txt          


In [0]:
import re

In [0]:
texts = open('habr_texts.txt').read().splitlines()
texts = [re.sub('[A-Za-z0-9]+', '', text) for text in texts]
texts = [tokenize(remove_tags(text.lower())) for text in texts]
texts = opt_normalize(texts, 3000)

In [0]:
def make_ngrams(text):
   ngrams = []
   new_text = [word for word in text if len(word) > 1]
   try: 
       for i in range(len(new_text)-1):
         ngrams.append(new_text[i] + '+' + new_text[i+1])
   except:
       pass
   return ngrams

In [0]:
ngrammed_texts = [make_ngrams(text) for text in texts if make_ngrams(text)]

In [0]:
print(ngrammed_texts[5])

['наш+литр', 'литр+число', 'число+метр', 'метр+наш', 'наш+год', 'год+год', 'год+наш', 'наш+метр', 'метр+наш', 'наш+наш', 'наш+литр', 'литр+плата', 'плата+далее', 'далее+наш', 'наш+наш', 'наш+наш', 'наш+плата', 'плата+далее', 'далее+литр', 'литр+литр', 'литр+наш', 'наш+плата', 'плата+литр', 'литр+наш', 'наш+плата', 'плата+литр', 'литр+наш', 'наш+наш', 'наш+наш', 'наш+наш', 'наш+плата', 'плата+метр', 'метр+литр', 'литр+далее', 'далее+наш', 'наш+плата', 'плата+наш', 'наш+литр', 'литр+метр', 'метр+плата', 'плата+метр', 'метр+наш', 'наш+наш', 'наш+наш', 'наш+плата', 'плата+литр', 'литр+наш', 'наш+наш', 'наш+метр', 'метр+плата', 'плата+метр', 'метр+далее', 'далее+наш', 'наш+наш', 'наш+наш', 'наш+плата', 'плата+литр', 'литр+метр', 'метр+наш', 'наш+наш', 'наш+число', 'число+литр', 'литр+метр', 'метр+литр', 'литр+наш', 'наш+наш', 'наш+метр', 'метр+метр', 'метр+число', 'число+метр', 'метр+наш', 'наш+плата', 'плата+наш', 'наш+наш', 'наш+наш', 'наш+метр', 'метр+литр', 'литр+наш', 'наш+плата', 'пла

Не уверена, что с автором этой статьи всё в порядке, но теперь хотя бы в список входят слова, а не что попало

In [0]:
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.8) 
p = gensim.models.phrases.Phraser(ph)
nrammed_texts = p[texts]
dictinary = gensim.corpora.Dictionary(ngrammed_texts)
dictinary.filter_extremes(no_above=0.05, no_below=10)
dictinary.compactify()
corpus = [dictinary.doc2bow(text) for text in ngrammed_texts]

In [0]:
num_topics = [50, 100, 200]
n_eval = [0,1]
a = [0,1]
best_res = 0
best_params = {}
for n_topics in num_topics:
  for n in n_eval:
    for alpha in a:
      lda = gensim.models.LdaMulticore(corpus, n_topics, id2word=dictinary, eval_every=n, alpha=alpha)
      coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                texts=texts, 
                                                  dictionary=dictinary, coherence='c_v')
      if coherence_model_lda.get_coherence() > best_res:
        best_res = coherence_model_lda.get_coherence()
        best_model = lda

In [0]:
best_model.print_topics()

[(80,
  '0.002*"выборка" + 0.002*"бот" + 0.002*"•" + 0.002*"вакансия" + 0.002*"студент" + 0.002*"инцидент" + 0.002*"атрибут" + 0.002*"преобразование" + 0.002*"координата" + 0.002*"боль"'),
 (70,
  '0.003*"вакансия" + 0.003*"атрибут" + 0.002*"выборка" + 0.002*"домен" + 0.002*"студент" + 0.002*"метка" + 0.002*"•" + 0.002*"фильм" + 0.002*"кампания" + 0.002*"координата"'),
 (37,
  '0.004*"боль" + 0.004*"•" + 0.002*"атрибут" + 0.002*"домен" + 0.002*"бот" + 0.002*"преобразование" + 0.002*"прибор" + 0.002*"выборка" + 0.002*"координата" + 0.002*"печать"'),
 (75,
  '0.005*"атрибут" + 0.003*"•" + 0.003*"боль" + 0.002*"метаданные" + 0.002*"студент" + 0.002*"вакансия" + 0.002*"координата" + 0.002*"генератор" + 0.002*"репликация" + 0.002*"прибор"'),
 (6,
  '0.003*"студент" + 0.002*"атрибут" + 0.002*"•" + 0.002*"вакансия" + 0.002*"инцидент" + 0.002*"боль" + 0.002*"кампания" + 0.002*"бот" + 0.002*"предприятие" + 0.002*"домен"'),
 (29,
  '0.002*"студент" + 0.002*"•" + 0.002*"атрибут" + 0.002*"вакансия

Ерунда какая-то: практически везде боль и студенты. Кажется, надо убрать точки из словаря и перезапустить подбор параметров для биграм. Кажется, также надо искусственным образом убрать студентов и вакансии, потому что студенты могут изучать что угодно и вакансии могут быть какие угодно. Вообще-то боль тоже стоит убрать, но очень хочется снова сгруппировать её с правительством.

In [0]:
def add_stopwords_after_preprocessing(texts, new_stopwords):
  new_texts = []
  for text in texts:
    new_text = []
    for word in text:
      if word not in new_stopwords:
        new_text.append(word)
    if new_text:
      new_texts.append(new_text)
  return new_texts

In [0]:
texts = add_stopwords_after_preprocessing(texts, ['.', 'студент', 'вакансия'])

In [0]:
#Промежуточный вывод: идея перебирать 300 с лишним сочетаний параметров очень плохая. Пока что по первым 20 примерам наблюдается следующее: обычно с регуляризацией результат на несколько десятых лучше, 
# но выбиваются два набора параметров, с которыми когерентность раза в два больше обычной, причём оба без регуляризации

In [0]:
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.8) 
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

In [0]:
tr = [-1, -0, 1]
no_above_values = [0.05, 0.15, 0.25]
no_below_values = [10, 25, 50]
num_topics = [50, 100, 200]
n_eval = [0,1]
a = [0,1]
best_res = 0
best_params = {}
for no_above in no_above_values:
  for no_below in no_below_values:
    for t in tr:
      for n in num_topics:
        for n_ev in n_eval:
          for alpha in a:
            ph = gensim.models.Phrases(ngrammed_texts, scoring='npmi', threshold = t)
            p = gensim.models.phrases.Phraser(ph)
            ngrammed_texts = p[ngrammed_texts]
            dictinary = gensim.corpora.Dictionary(ngrammed_texts)
            dictinary.filter_extremes(no_above=no_above, no_below=no_below)
            dictinary.compactify()
            corpus = [dictinary.doc2bow(text) for text in texts]
            lda = gensim.models.LdaMulticore(corpus, n, id2word=dictinary, eval_every=n_ev, alpha=alpha)
            coherence_model_lda = gensim.models.CoherenceModel(model=lda, texts=ngrammed_texts, dictionary=dictinary, coherence='c_v')
            params = [t, no_above, no_below, n, n_ev, alpha]
            c = coherence_model_lda.get_coherence() 
            print('cogerence=', c, 'params:', params)
            if c > best_res:
              best_params_model = lda

cogerence= 0.3113832324845635 params: [-1, 0.05, 10, 50, 0, 0]
cogerence= 0.37067050066018337 params: [-1, 0.05, 10, 50, 0, 1]
cogerence= 0.3056127437967403 params: [-1, 0.05, 10, 50, 1, 0]
cogerence= 0.3709433252914618 params: [-1, 0.05, 10, 50, 1, 1]
cogerence= 0.3133048203514031 params: [-1, 0.05, 10, 100, 0, 0]
cogerence= 0.3886867147612275 params: [-1, 0.05, 10, 100, 0, 1]
cogerence= 0.30753516239835954 params: [-1, 0.05, 10, 100, 1, 0]
cogerence= 0.3847320485348156 params: [-1, 0.05, 10, 100, 1, 1]
cogerence= 0.6618007637492457 params: [-1, 0.05, 10, 200, 0, 0]
cogerence= 0.3919988866546882 params: [-1, 0.05, 10, 200, 0, 1]
cogerence= 0.6618007637492457 params: [-1, 0.05, 10, 200, 1, 0]
cogerence= 0.3964581558850766 params: [-1, 0.05, 10, 200, 1, 1]
cogerence= 0.32360493329650614 params: [0, 0.05, 10, 50, 0, 0]
cogerence= 0.36355462406373285 params: [0, 0.05, 10, 50, 0, 1]
cogerence= 0.3129983890656569 params: [0, 0.05, 10, 50, 1, 0]
cogerence= 0.37137854092165135 params: [0, 0.0

KeyboardInterrupt: ignored

In [0]:
#полный перебор не выполнился, возьмём один из лучших опробованных наборов параметров
[t, no_above, no_below, n, n_ev, alpha] = [-1, 0.05, 10, 200, 1, 0]
ph = gensim.models.Phrases(ngrammed_texts, scoring='npmi', threshold = t)
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[ngrammed_texts]
dictinary = gensim.corpora.Dictionary(ngrammed_texts)
dictinary.filter_extremes(no_above=no_above, no_below=no_below)
dictinary.compactify()
corpus = [dictinary.doc2bow(text) for text in texts]
lda = gensim.models.LdaMulticore(corpus, n, id2word=dictinary, eval_every=n_ev, alpha=alpha)

In [0]:
lda.print_topics()

[(172,
  'nan*"медик" + nan*"масса_килограмм" + nan*"невольно" + nan*"находиться_прямо" + nan*"насквозь" + nan*"написать_очень" + nan*"молоток" + nan*"минь" + nan*"некоторый_модель" + nan*"материя"'),
 (42,
  'nan*"медик" + nan*"масса_килограмм" + nan*"невольно" + nan*"находиться_прямо" + nan*"насквозь" + nan*"написать_очень" + nan*"молоток" + nan*"минь" + nan*"некоторый_модель" + nan*"материя"'),
 (188,
  'nan*"медик" + nan*"масса_килограмм" + nan*"невольно" + nan*"находиться_прямо" + nan*"насквозь" + nan*"написать_очень" + nan*"молоток" + nan*"минь" + nan*"некоторый_модель" + nan*"материя"'),
 (136,
  'nan*"медик" + nan*"масса_килограмм" + nan*"невольно" + nan*"находиться_прямо" + nan*"насквозь" + nan*"написать_очень" + nan*"молоток" + nan*"минь" + nan*"некоторый_модель" + nan*"материя"'),
 (2,
  'nan*"медик" + nan*"масса_килограмм" + nan*"невольно" + nan*"находиться_прямо" + nan*"насквозь" + nan*"написать_очень" + nan*"молоток" + nan*"минь" + nan*"некоторый_модель" + nan*"материя"')

По-прежнему получаются одинаковые наборы слов. Наверно, нужно получать биграммы адекватным способом, а не склеивать в псевдослова

In [0]:
texts = open('habr_texts.txt').read().splitlines()
texts = [re.sub('[A-Za-z0-9]+', '', text) for text in texts]
texts = [tokenize(remove_tags(text.lower())) for text in texts]
texts = opt_normalize(texts, 30000)

In [0]:
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4)
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]
dictinary = gensim.corpora.Dictionary(ngrammed_texts)
dictinary.filter_extremes(no_above=0.05, no_below=10)
dictinary.compactify()
corpus = [dictinary.doc2bow(text) for text in ngrammed_texts]

In [0]:
best_res = 0
for alpha in [0,1]:
  for n_eval in [0, 1]:
    lda = gensim.models.LdaMulticore(corpus, n, id2word=dictinary, eval_every=n_ev, alpha=alpha)
    coherence_model_lda = gensim.models.CoherenceModel(model=lda, texts=ngrammed_texts, dictionary=dictinary, coherence='c_v')
    if coherence_model_lda.get_coherence() > best_res:
      best_model = lda
      best_params = [alpha, n_eval]
print(best_params)

[1, 1]


In [0]:
best_model.print_topics()

[(25,
  '0.008*"кампания" + 0.008*"боль" + 0.006*"атрибут" + 0.005*"репликация" + 0.004*"усилитель" + 0.004*"координата" + 0.003*"младенец" + 0.003*"объявление" + 0.003*"•" + 0.003*"спецификация"'),
 (148,
  '0.009*"боль" + 0.007*"атрибут" + 0.004*"кампания" + 0.004*"•" + 0.004*"объявление" + 0.004*"младенец" + 0.004*"репликация" + 0.003*"прибор" + 0.003*"усилитель" + 0.003*"сопротивление"'),
 (20,
  '0.009*"боль" + 0.007*"кампания" + 0.005*"атрибут" + 0.004*"метаданные" + 0.004*"координата" + 0.003*"спецификация" + 0.003*"сопротивление" + 0.003*"объявление" + 0.003*"•" + 0.003*"усилитель"'),
 (35,
  '0.009*"боль" + 0.007*"атрибут" + 0.006*"кампания" + 0.005*"репликация" + 0.004*"координата" + 0.004*"метаданные" + 0.003*"печать" + 0.003*"объявление" + 0.003*"усилитель" + 0.003*"головка"'),
 (152,
  '0.007*"боль" + 0.007*"атрибут" + 0.005*"кампания" + 0.004*"усилитель" + 0.004*"печать" + 0.004*"головка" + 0.004*"координата" + 0.003*"метаданные" + 0.003*"объявление" + 0.003*"репликация"'

Темы получились очень похожие и не особо интепретитуемые
(25,
  '0.008*"кампания" + 0.008*"боль" + 0.006*"атрибут" + 0.005*"репликация" + 0.004*"усилитель" + 0.004*"координата" + 0.003*"младенец" + 0.003*"объявление" + 0.003*"•" + 0.003*"спецификация"'),
 (148,
  '0.009*"боль" + 0.007*"атрибут" + 0.004*"кампания" + 0.004*"•" + 0.004*"объявление" + 0.004*"младенец" + 0.004*"репликация" + 0.003*"прибор" + 0.003*"усилитель" + 0.003*"сопротивление"'),
 (20,
  '0.009*"боль" + 0.007*"кампания" + 0.005*"атрибут" + 0.004*"метаданные" + 0.004*"координата" + 0.003*"спецификация" + 0.003*"сопротивление" + 0.003*"объявление" + 0.003*"•" + 0.003*"усилитель"'),
  Предполагаю, что тексты хабра недостаточно разнообразны по тематике, чтобы через нграммы разграничивать разные значения одного слова

Добавим tfidf

In [0]:
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.8) 
p = gensim.models.phrases.Phraser(ph)
nrammed_texts = p[texts]
dictionary = gensim.corpora.Dictionary(ngrammed_texts)
dictionary.filter_extremes(no_above=0.05, no_below=10)
dictionary.compactify()
corpus = [dictionary.doc2bow(text) for text in ngrammed_texts]

In [0]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)

In [0]:
n_eval = [0,1]
alpha = [0,1,2]
best_res = 0
for n in n_eval:
  for a in alpha:
    lda = gensim.models.LdaMulticore(corpus=tfidf[corpus], id2word=dictionary, eval_every=n)
    coherence_model_lda = gensim.models.CoherenceModel(model=lda, texts=texts, dictionary=dictionary, coherence='c_v')
    c = coherence_model_lda.get_coherence() 
    print('Когерентность:', c, 'параметры:', [n, a])
    if c > best_res:
      best_model = lda

Когерентность: nan параметры: [0, 0]
Когерентность: nan параметры: [0, 1]
Когерентность: nan параметры: [0, 2]
Когерентность: nan параметры: [1, 0]
Когерентность: nan параметры: [1, 1]
Когерентность: nan параметры: [1, 2]


По метрикам все модели не очень, посмотрим н последнюю

In [0]:
lda.print_topics()

[(3,
  '0.123*"датчик" + 0.091*"станция" + 0.045*"переходить" + 0.024*"авторизация" + 0.024*"кластер" + 0.023*"убедиться" + 0.023*"софт" + 0.022*"обслуживание" + 0.022*"отсутствовать" + 0.022*"корректно"'),
 (40,
  '0.034*"дерево" + 0.021*"субд" + 0.020*"датчик" + 0.019*"полый" + 0.017*"шифрование" + 0.017*"тело" + 0.015*"граница" + 0.015*"клетка" + 0.014*"ося" + 0.011*"оперативный"'),
 (60,
  '0.084*"клетка" + 0.044*"курс" + 0.026*"рейтинг" + 0.025*"баг" + 0.024*"штука" + 0.024*"ценность" + 0.024*"внезапно" + 0.023*"тело" + 0.023*"рассказывать" + 0.023*"пк"'),
 (5,
  '0.036*"плагин" + 0.028*"слой" + 0.021*"доклад" + 0.021*"бд" + 0.012*"кат" + 0.011*"ия" + 0.009*"угол" + 0.009*"допустить" + 0.009*"аппарат" + 0.009*"«как"'),
 (76,
  '0.022*"спутник" + 0.018*"“_”" + 0.014*"регистрация" + 0.014*"желательно" + 0.014*"отображение" + 0.012*"программный" + 0.012*"мс" + 0.012*"блокчейн" + 0.011*"драйвер" + 0.011*"директория"'),
 (54,
  '0.017*"бд" + 0.016*"улучшение" + 0.016*"тело" + 0.015*"ия

Чтобы это проинтерпретировать, надо постараться.
Можно притянуть за уши что-то такое:
- подготовительная стадия (кто-то рзвивается, изучает синтаксис, собирается сменить сферу, делает что-то медленно):
  '0.208*"спецификация" + 0.048*"настоящее" + 0.018*"синтаксис" + 0.018*"объём" + 0.018*"флаг" + 0.018*"изучить" + 0.018*"развиваться" + 0.018*"предоставить" + 0.017*"медленно" + 0.017*"собираться"')
  - получение данных в ходе экспериментов (исследователь в лаборатории занимается какими-то преобразованиями и представляет результаты на рисунках)
  - организация работы (бюджет, исполнения, эксплуатация, аналитика, январь - видимо, как срок сдачи отчётности)

  После добавления tfidf выделяются подтемы внутри основной тематики текстов, а не прихватываются периферийные (боль, политика и т.п.)
  Сами тематики лучше, чем без tfidf, но по-прежнему хуже, чем без нграм.
  

Воспользуемся NFM

In [0]:

from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [0]:
stexts = [' '.join(text) for text in texts]

In [0]:
vectorizer = TfidfVectorizer(max_features=1000, min_df=10, max_df=0.3, ngram_range=(1,3))
X = vectorizer.fit_transform(stexts)

In [28]:
import math
min_err = math.inf
for n in [15, 30, 45]:
  for a in [0, 1, 2]:
    model = NMF(n_components=30, alpha=a)  
    model.fit(X)
    err = model.reconstruction_err_
    print('err:', err, 'params:', [n, a])
    if err < min_err:
      min_err = err
      best_model = model


err: 54.519443316939274 params: [15, 0]
err: 54.80389249524452 params: [15, 1]
err: 55.5789250163123 params: [15, 2]
err: 54.51074161526118 params: [30, 0]
err: 54.77937362336289 params: [30, 1]
err: 55.578926043744595 params: [30, 2]
err: 54.52897392816625 params: [45, 0]
err: 54.77441694610429 params: [45, 1]
err: 55.57700716896436 params: [45, 2]


In [30]:
feat_names = vectorizer.get_feature_names()
top_words = best_model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 какой то  ещё  жизнь  вообще
1 игра  игрок  игровой  играть
2 файл  папка  строка  скрипт
3 сервер  настройка  сервис  конфигурация
4 устройство  смартфон  мобильный  устройство который
5 объект  класс  метод  значение
6 язык  программирование  язык программирование  программист
7 сайт  страница  браузер  реклама
8 память  процессор  гб  диск
9 продукт  сервис  технология  платформа
10 блок  переменный  длина  кнопка
11 мозг  исследование  учёный  группа
12 элемент  компонент  значение  свойство
13 тест  тестирование  библиотека  ошибка
14 сеть  трафик  связь  интернет
15 запрос  таблица  строка  ответ
16 энергия  земля  вселенная  поверхность
17 сообщение  бот  сервис  канал
18 клиент  услуга  сотрудник  бизнес
19 модель  обучение  ия  изображение
20 машина  виртуальный  виртуальный машина  вм
21 модуль  пакет  компонент  поддержка
22 сигнал  звук  частота  значение
23 книга  часы  друг  читать
24 камера  видео  смартфон  карта
25 программа  компьютер  программный  программный обесп

Группы хорошо интерпретируются:
- работа с базами данных (запрос  таблица  строка  ответ)
- организация бизнеса (клиент услуга сотрудник бизнес)
- обработка звука (сигнал звук частота значение)
Но некоторые классы представлены однокоренными словами (игра игрок игровой играть), а некоторые дублируются
Возможно, при переборе моделей стоит извлекать слова в каждом результате, выявлять такие случаи и не записывать в best model варианты с дублями/пробовать уменьшать количество тем при тех же остальных параметрах. Как ни странно, от количества тем метрика ошибки почти не зависела